In [1]:
import pandas as pd

from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise import accuracy

## Loading Data

In [2]:
eventDataSet = pd.read_csv('data/FilteredEvents/capstone_event_2018_07.csv')
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_08.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_09.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_10.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_11.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2018_12.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_01.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_02.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_03.csv'))
eventDataSet = eventDataSet.append(pd.read_csv('data/FilteredEvents/capstone_event_2019_04.csv'))

In [3]:
eventDataSet = eventDataSet.replace(to_replace=["Kiralık", "Satılik", "Satılık"], value = ["Kiralik", "Satilik", "Satilik"])
eventDataSet = eventDataSet[(eventDataSet.districtName.notnull())]

salesEventDataSet = eventDataSet[(eventDataSet.listingTypeId == 1)]
rentEventDataSet = eventDataSet[(eventDataSet.listingTypeId == 2)]

In [4]:
ds = rentEventDataSet[(rentEventDataSet.cityId == 1) & (rentEventDataSet.subPropertyType.notnull()) & (rentEventDataSet.subPropertyType == 'Daire') & (rentEventDataSet.listingId.notnull())]
ds = ds.filter(['event_date', 'event_timestamp', 'user_pseudo_id', 'event_name', 'listingId'])

ds.head()

,event_date,event_timestamp,user_pseudo_id,event_name,listingId
133,20180701,1530476403832031,804947ac621b45f4994b01c2026a6d5c,ListingView,1464629.0
134,20180701,1530476495590000,804947ac621b45f4994b01c2026a6d5c,ListingView,1440722.0
135,20180701,1530476527232005,804947ac621b45f4994b01c2026a6d5c,ListingView,1110019.0
136,20180701,1530476727967011,804947ac621b45f4994b01c2026a6d5c,ListingView,1132440.0
137,20180701,1530476791836016,804947ac621b45f4994b01c2026a6d5c,ListingView,1254301.0


In [5]:
listingDataSet = pd.read_csv('data/listing/listing0.csv')

for i in range(1, 668):
    filePath = 'data/listing/listing'+str(i)+'.csv'
    listingDataSet = listingDataSet.append(pd.read_csv(filePath))

In [6]:
bigDS = ds.set_index('listingId').join(listingDataSet.set_index('id'))
bigDS.head()

,event_date,event_timestamp,user_pseudo_id,event_name,no,listingType,type,parentType,price,currency,...,roomCount,livingRoomCount,roomSlug,size,cityId,cityName,countyId,countyName,districtId,districtName
4629.0,20190301,1551388568096023,858aa504d1c8c13d18127d1dbaa942fb,ListingView,593068.0,Rent,Daire,Konut,5750.0,TRY,...,3.0,1.0,3 + 1,140.0,1.0,İstanbul,903.0,Tuzla,907.0,İstasyon
4629.0,20190418,1555619270058023,858aa504d1c8c13d18127d1dbaa942fb,ListingView,593068.0,Rent,Daire,Konut,5750.0,TRY,...,3.0,1.0,3 + 1,140.0,1.0,İstanbul,903.0,Tuzla,907.0,İstasyon
4629.0,20190427,1556355770050023,858aa504d1c8c13d18127d1dbaa942fb,ListingView,593068.0,Rent,Daire,Konut,5750.0,TRY,...,3.0,1.0,3 + 1,140.0,1.0,İstanbul,903.0,Tuzla,907.0,İstasyon
52487.0,20190301,1551472313903033,edabfe954f426dbe31132823e4122474,ListingView,268896.0,Rent,Daire,Konut,900.0,TRY,...,2.0,1.0,2 + 1,115.0,1.0,İstanbul,507.0,Gaziosmanpaşa,509.0,Karadeniz
54773.0,20190318,1552907050757024,69ac0e82545dfbacb2318515f78801d8,ListingView,470169.0,Rent,Daire,Konut,2750.0,TRY,...,2.0,1.0,2 + 1,85.0,1.0,İstanbul,448.0,Fatih,461.0,Binbirdirek


In [7]:
kkd = bigDS

In [8]:
bigDS = kkd
bigDS["interest"] = bigDS.cityName + "_" + bigDS.countyName + "_" + bigDS.districtName + "_" + (round(bigDS.price / 1000)).astype(str)
bigDS = bigDS.filter(['user_pseudo_id', 'interest'])
bigDS['count'] = 0
ratingDS = bigDS.groupby(['user_pseudo_id', 'interest'])['count'].count().reset_index()
ratingDS.head()


,user_pseudo_id,interest,count
0,00001238ac28333afc87b94a992f0319,İstanbul_Pendik_Esenyalı_1.0,1
1,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Aydınlı_1.0,2
2,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Aydıntepe_1.0,1
3,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Mimar Sinan_1.0,2
4,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Orta_1.0,1


In [9]:
userSumListing = ratingDS.groupby(['user_pseudo_id'])['count'].max().reset_index()
userSumListing.columns = ['user_pseudo_id', 'maxCount']
userSumListing.head()


,user_pseudo_id,maxCount
0,00001238ac28333afc87b94a992f0319,2
1,00016bc8bc90bad8001f6b4e64b7f43b,7
2,0001A4ABCCD44496893DD404253E706C,2
3,00031b94b31e128718d3c8da09b113d8,4
4,0004B670A01C4D109162BF2C45E10050,3


In [10]:
rating = pd.concat([ratingDS.set_index('user_pseudo_id'), userSumListing.set_index('user_pseudo_id')], axis=1,
                   join='inner').reset_index()
rating['rating'] = round(3 * rating['count'] / rating['maxCount']) + 2

final = rating.filter(['user_pseudo_id', 'interest', 'rating'])
final.head()

,user_pseudo_id,interest,rating
0,00001238ac28333afc87b94a992f0319,İstanbul_Pendik_Esenyalı_1.0,4.0
1,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Aydınlı_1.0,5.0
2,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Aydıntepe_1.0,4.0
3,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Mimar Sinan_1.0,5.0
4,00001238ac28333afc87b94a992f0319,İstanbul_Tuzla_Orta_1.0,4.0


In [11]:
reader = Reader()
data = Dataset.load_from_df(final[['user_pseudo_id', 'interest', 'rating']], reader)

In [12]:
trainset, testset = train_test_split(data, test_size=0.80)
algo = KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1550


1.155020630089364

In [17]:
id = algo.trainset.to_inner_iid('İstanbul_Kağıthane_Nurtepe_1.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Kağıthane_Nurtepe_1.0****
İstanbul_Sultanbeyli_Battalgazi_1.0
İstanbul_Kağıthane_Çeliktepe_2.0
İstanbul_Beylikdüzü_Cumhuriyet_4.0
İstanbul_Kadıköy_Merdivenköy_1.0
İstanbul_Sarıyer_Yeniköy_2.0


In [18]:
id = algo.trainset.to_inner_iid('İstanbul_Kartal_Çavuşoğlu_1.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Kartal_Çavuşoğlu_1.0****
İstanbul_Üsküdar_Ferah_1.0
İstanbul_Çekmeköy_Aydınlar_1.0
İstanbul_Sultanbeyli_Battalgazi_1.0
İstanbul_Büyükçekmece_Mimaroba_2.0
İstanbul_Zeytinburnu_Çırpıcı_1.0


In [19]:
id = algo.trainset.to_inner_iid('İstanbul_Çekmeköy_Aydınlar_1.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Çekmeköy_Aydınlar_1.0****
İstanbul_Üsküdar_Ferah_1.0
İstanbul_Beykoz_Göztepe_2.0
İstanbul_Üsküdar_Kirazlıtepe_1.0
İstanbul_Eyüpsultan_Çırçır_1.0
İstanbul_Maltepe_Küçükyalı_2.0


In [20]:
id = algo.trainset.to_inner_iid('İstanbul_Maltepe_Küçükyalı_2.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Maltepe_Küçükyalı_2.0****
İstanbul_Çekmeköy_Aydınlar_1.0
İstanbul_Üsküdar_Kirazlıtepe_1.0
İstanbul_Zeytinburnu_Çırpıcı_1.0
İstanbul_Beyoğlu_Fetihtepe_1.0
İstanbul_Çekmeköy_Çamlık_1.0


In [21]:
id = algo.trainset.to_inner_iid('İstanbul_Maltepe_Küçükyalı_3.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Maltepe_Küçükyalı_3.0****
İstanbul_Esenler_Çifte Havuzlar_1.0
İstanbul_Büyükçekmece_Fatih_2.0
İstanbul_Beyoğlu_Müeyyetzade_8.0
İstanbul_Bakırköy_Kartaltepe_3.0
İstanbul_Beylikdüzü_Yakuplu_0.0


In [27]:
id = algo.trainset.to_inner_iid('İstanbul_Maltepe_Küçükyalı_1.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Maltepe_Küçükyalı_1.0****
İstanbul_Maltepe_Girne_1.0
İstanbul_Beyoğlu_Kocatepe_2.0
İstanbul_Ataşehir_İçerenköy_1.0
İstanbul_Beşiktaş_Abbasağa_2.0
İstanbul_Esenler_Birlik_0.0


In [26]:
id = algo.trainset.to_inner_iid('İstanbul_Kadıköy_Caferağa_3.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Kadıköy_Caferağa_3.0****
İstanbul_Avcılar_Denizköşkler_2.0
İstanbul_Beşiktaş_Cihannüma_2.0
İstanbul_Bağcılar_İnönü_1.0
İstanbul_Sarıyer_Ferahevler_18.0
İstanbul_Eyüpsultan_Güzeltepe_2.0


In [24]:
id = algo.trainset.to_inner_iid('İstanbul_Beşiktaş_Bebek_3.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Beşiktaş_Bebek_3.0****
İstanbul_Kadıköy_19 Mayıs_2.0
İstanbul_Bahçelievler_Şirinevler_2.0
İstanbul_Şişli_Meşrutiyet_2.0
İstanbul_Kadıköy_Caferağa_2.0
İstanbul_Esenler_Havaalanı_1.0


In [25]:
id = algo.trainset.to_inner_iid('İstanbul_Şişli_Meşrutiyet_2.0')

print("****" + algo.trainset.to_raw_iid(id) + "****")
neighbors = algo.get_neighbors(id, k=5)
neighbors = (algo.trainset.to_raw_iid(id) for id in neighbors)

for n in neighbors:
    print(n)

****İstanbul_Şişli_Meşrutiyet_2.0****
İstanbul_Beyoğlu_Yahya Kahya_1.0
İstanbul_Beyoğlu_Kocatepe_2.0
İstanbul_Beşiktaş_Bebek_3.0
İstanbul_Esenler_Havaalanı_1.0
İstanbul_Arnavutköy_Mavigöl_1.0
